## Assign equipment category to exercises

In [12]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from openai import OpenAI
import re
from dotenv import load_dotenv

In [13]:
_ = load_dotenv()

In [6]:
dataframes = {}
data_folder = '../data_processed/'
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        dataframes[filename[:-4]] = df
for name, df in dataframes.items():
    print(f"Columns in dataframe '{name}':")
    print(df.columns)
    print()
exercises = dataframes['exercises_cleaned']
null_values_exercises = exercises.isnull().sum()
print("\nNull values in exercises dataframe:")
print(null_values_exercises)

/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_25206/2847802780.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_25206/2847802780.py:6: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_25206/2847802780.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/3f/hpgbw8dd459305qvgctxmywc0000gn/T/ipykernel_25206/2847802780.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Columns in dataframe 'processed_exercises_v2':
Index(['CloudId', 'Gender', 'Age', 'PhysicalActivityMacroTypeName',
       'ExerciseName', 'EquipmentName', 'DoneOnUTC', 'Duration_sec',
       'Calories', 'MetsMin', 'ExerciseBodyPartName', 'ExerciseMusclesName',
       'METs', 'EquipmentCategory', 'Week', 'Year', 'Date'],
      dtype='object')

Columns in dataframe 'body_part_categorized':
Index(['Leg Curl Biostr:LowerBody'], dtype='object')

Columns in dataframe 'exercises_cleaned':
Index(['CloudId', 'Gender', 'Age', 'PhysicalActivityMacroTypeName',
       'ExerciseName', 'EquipmentName', 'DoneOnUTC', 'Duration_sec',
       'Calories', 'MetsMin', 'ExerciseBodyPartName', 'ExerciseMusclesName',
       'METs'],
      dtype='object')

Columns in dataframe 'biometrics_cleaned':
Index(['CloudId', 'Gender', 'Age', 'BiometricName', 'MeasureProvidedBy',
       'MeasuredOnUTC', 'Value'],
      dtype='object')

Columns in dataframe 'workout_metrics':
Index(['CloudId', 'Date', 'WorkoutId', 'total_w

In [5]:
# Count the number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null
null_bodypart_muscle_count = exercises[(exercises['ExerciseBodyPartName'].isnull()) & (exercises['ExerciseMusclesName'].isnull())].shape[0]
print(f"Number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null: {null_bodypart_muscle_count}")
# Count the number of rows where either ExerciseBodyPartName or ExerciseMusclesName is null
null_bodypart_or_muscle_count = exercises[(exercises['ExerciseBodyPartName'].isnull()) | (exercises['ExerciseMusclesName'].isnull())].shape[0]
print(f"Number of rows where either ExerciseBodyPartName or ExerciseMusclesName are null: {null_bodypart_or_muscle_count}")

Number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null: 862729
Number of rows where either ExerciseBodyPartName or ExerciseMusclesName are null: 863022


In [6]:
# Filter the exercises DataFrame for rows where both ExerciseBodyPartName and ExerciseMusclesName are null
null_bodypart_muscle_examples = exercises[(exercises['ExerciseBodyPartName'].isnull()) & (exercises['ExerciseMusclesName'].isnull())]
null_bodypart_muscle_examples['EquipmentName'].value_counts().head(15)

EquipmentName
Dumbbells               53564
Body weight             42543
Run Artis               23033
Chest Press Biostr      22352
Run                     21355
Low Row Biostr          21338
Bike Artis              20266
Vertical Tr. Biostr     20230
Total Abdom. Biostr     20145
Barbell                 19147
Chest Press Sel         17784
Leg Press Sel           17713
Leg Extension Biostr    17268
Leg Curl Biostr         15880
Bike                    14779
Name: count, dtype: int64

***Replace Bike Artis with Bike and assign it to lowerbody and Run Artis with Run and assign it to totalbody***

In [24]:
# Replace 'Run Artis' with 'Run' and 'Bike Artis' with 'Bike' in the EquipmentName column
exercises['EquipmentName'] = exercises['EquipmentName'].replace({'Run Artis': 'Run', 'Bike Artis': 'Bike'})

# Update ExerciseBodyPartName based on the EquipmentName
exercises.loc[exercises['EquipmentName'] == 'Run', 'ExerciseBodyPartName'] = 'TotalBody'
exercises.loc[exercises['EquipmentName'] == 'Bike', 'ExerciseBodyPartName'] = 'LowerBody'

In [ ]:
# Count the number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null
null_bodypart_muscle_count = exercises[(exercises['ExerciseBodyPartName'].isnull()) & (exercises['ExerciseMusclesName'].isnull())].shape[0]
print(f"Number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null: {null_bodypart_muscle_count}")
# Filter the exercises DataFrame for rows where both ExerciseBodyPartName and ExerciseMusclesName are null
null_bodypart_muscle_examples = exercises[(exercises['ExerciseBodyPartName'].isnull()) & (exercises['ExerciseMusclesName'].isnull())]
null_bodypart_muscle_examples['EquipmentName'].value_counts().head(15)

Number of rows where both ExerciseBodyPartName and ExerciseMusclesName are null: 783296


EquipmentName
Dumbbells               53564
Body weight             42543
Chest Press Biostr      22352
Low Row Biostr          21338
Vertical Tr. Biostr     20230
Total Abdom. Biostr     20145
Barbell                 19147
Chest Press Sel         17784
Leg Press Sel           17713
Leg Extension Biostr    17268
Leg Curl Biostr         15880
Leg Extension Sel       14437
Lat Machine Sel         14324
Stretching              13503
Abdominal Crunch Sel    12785
Name: count, dtype: int64

### Equipment

In [38]:
exercises['EquipmentName'].nunique()

370

There are 370 distinct equipment names

In [26]:
equipment_names_str = ', '.join(exercises['EquipmentName'].dropna().unique())
print(equipment_names_str)

Bike, Run, Leg Curl Biostr, Leg Press Biostr, Leg Extension Sel, Cables Crossed Elt+, Dumbbells, Barbell, Body weight, Kettlebell, Wellness Ball, Climb Excite 1000, Skillrow, Skillrun, Chest Press Sel, Vertical Tr. Artis, Total Abd. Artis, Abductor Artis, Synchro Excite 1000, Climb Excite Live, Tubes, Cable Stations 4, Row Pure, Chest Press Pure, Run Excite 1000, Synchro Excite 700, Arm Extension Sel, Synchro Artis, Bike Excite 1000, Run Excite Live, Rear Kick Pure, Leg Press Sel, Total Abdominal Sel, Lower Back Sel, Synchro Artis 2019, Multipower, Pectoral Sel, Shoulder Press Sel, Anterior Flexability, Run Excite 600, Pulley Sel, Cables Crossed Sel, Total Abdom. Biostr, Vertical Tr. Biostr, Crossover Cables, Recline, Wave, Lat Machine Sel, Arm Extension Artis, Shoulder Press Artis, Low Row Artis, Cable Jungle, Lower Back Bench, Leg Curl Elt+, Leg Curl Sel, Adductor Artis, Chest Press Biostr, Low Row Biostr, Step/Squat Kst, High Pull Kst, Leg Curl Artis, Chest Press Artis, Adductor Sel

In [96]:
equipment_names_str

'Bike, Run, Leg Curl Biostr, Leg Press Biostr, Leg Extension Sel, Cables Crossed Elt+, Dumbbells, Barbell, Body weight, Kettlebell, Wellness Ball, Climb Excite 1000, Skillrow, Skillrun, Chest Press Sel, Vertical Tr. Artis, Total Abd. Artis, Abductor Artis, Synchro Excite 1000, Climb Excite Live, Tubes, Cable Stations 4, Row Pure, Chest Press Pure, Run Excite 1000, Synchro Excite 700, Arm Extension Sel, Synchro Artis, Bike Excite 1000, Run Excite Live, Rear Kick Pure, Leg Press Sel, Total Abdominal Sel, Lower Back Sel, Synchro Artis 2019, Multipower, Pectoral Sel, Shoulder Press Sel, Anterior Flexability, Run Excite 600, Pulley Sel, Cables Crossed Sel, Total Abdom. Biostr, Vertical Tr. Biostr, Crossover Cables, Recline, Wave, Lat Machine Sel, Arm Extension Artis, Shoulder Press Artis, Low Row Artis, Cable Jungle, Lower Back Bench, Leg Curl Elt+, Leg Curl Sel, Adductor Artis, Chest Press Biostr, Low Row Biostr, Step/Squat Kst, High Pull Kst, Leg Curl Artis, Chest Press Artis, Adductor Se

In [111]:
# Split the string by commas and count the number of resulting values
num_values = len(equipment_names_str.split(','))
print(f"The equipment_names_str string has {num_values} values.")

The equipment_names_str string has 370 values.


In [130]:
# Split the string by commas to get a list of equipment names
equipment_list = equipment_names_str.split(', ')

Functional training refers to a style of exercise that focuses on improving your ability to perform everyday movements more effectively and safely. It emphasizes exercises that mimic real-life activities and engage multiple muscle groups and joints, making your body stronger and more coordinated for daily tasks, sports, or physically demanding activities.

In [ ]:
openaikey = os.getenv('OPENAIKEY')
client = OpenAI(api_key=openaikey)

# Function to split a list into batches
def split_into_batches(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Configurable batch size
batch_size = 10

# Split the equipment_list into batches
batches = list(split_into_batches(equipment_list, batch_size))

# Initialize an empty string to store the concatenated response
full_response = ""

# Loop through each batch and make the API call
for batch in batches:
    equipment_names = ', '.join(batch)
    
    # Prepare the prompt for the OpenAI API
    prompt = "Categorize the following gym equipment into the following broader categories -Strength Training, Free Weights, Functional Training, Cardiovascular, Flexibility & Mobility-  Add everything only to one category. Dont skip any strings and dont change them at all from the way I give it to you. Give it in the form equip:category (next row)equip:category Don't write anything else. Also write each equipment seperately (next line)\n\n"
    prompt += equipment_names

    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful fitness data analysis assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        response = completion.choices[0].message.content
        print(response)
        full_response += response + "\n"
    except Exception as e:
        print(f"RateLimitError: {e}")

print(full_response)

Bike:Cardiovascular
Run:Cardiovascular
Leg Curl Biostr:Strength Training
Leg Press Biostr:Strength Training
Leg Extension Sel:Strength Training
Cables Crossed Elt+:Functional Training
Dumbbells:Free Weights
Barbell:Free Weights
Body weight:Functional Training
Kettlebell:Functional Training
Wellness Ball:Flexibility & Mobility
Climb Excite 1000:Cardiovascular
Skillrow:Cardiovascular
Skillrun:Cardiovascular
Chest Press Sel:Strength Training
Vertical Tr. Artis:Functional Training
Total Abd. Artis:Functional Training
Abductor Artis:Functional Training
Synchro Excite 1000:Cardiovascular
Climb Excite Live:Cardiovascular
Tubes, Cable Stations: Functional Training  
Row Pure: Strength Training  
Chest Press Pure: Strength Training  
Run Excite 1000: Cardiovascular  
Synchro Excite 700: Cardiovascular  
Arm Extension Sel: Strength Training  
Synchro Artis: Cardiovascular  
Bike Excite 1000: Cardiovascular  
Run Excite Live: Cardiovascular  
Rear Kick Pure:Functional Training
Leg Press Sel:Stren

In [132]:
# Write the string full_response to a file
with open('equip_categories.txt', 'w') as file:
    file.write(full_response)

In [ ]:
# Define the path to the file
file_path = 'data/equip_categories.txt'

# Initialize an empty dictionary to store the equipment-category map
equipment_category_map = {}

# Open and read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Process each line
for line in lines:
    line = line.strip()  # Remove extra spaces or newline characters
    if line:
        print(line)  # Check if the line is not empty
        equipment, category = line.rsplit(':', 1)
        equipment_category_map[equipment.strip()] = category.strip()

# Print or return the resulting map
print(equipment_category_map)


Bike:Cardiovascular
Run:Cardiovascular
Leg Curl Biostr:Strength Training
Leg Press Biostr:Strength Training
Leg Extension Sel:Strength Training
Cables Crossed Elt+:Functional Training
Dumbbells:Free Weights
Barbell:Free Weights
Body weight:Functional Training
Kettlebell:Functional Training
Wellness Ball:Flexibility & Mobility
Climb Excite 1000:Cardiovascular
Skillrow:Cardiovascular
Skillrun:Cardiovascular
Chest Press Sel:Strength Training
Vertical Tr. Artis:Functional Training
Total Abd. Artis:Functional Training
Abductor Artis:Functional Training
Synchro Excite 1000:Cardiovascular
Climb Excite Live:Cardiovascular
Tubes: Functional Training
Cable Stations: Functional Training
Cable Stations 4: Functional Training
Row Pure: Strength Training
Chest Press Pure: Strength Training
Run Excite 1000: Cardiovascular
Synchro Excite 700: Cardiovascular
Arm Extension Sel: Strength Training
Synchro Artis: Cardiovascular
Bike Excite 1000: Cardiovascular
Run Excite Live: Cardiovascular
Rear Kick Pure

In [162]:
num_keys = len(equipment_category_map)
print(f"The category_map dictionary has {num_keys} keys.")

The category_map dictionary has 372 keys.


In [163]:
# Get the set of unique equipment names from the exercises dataframe
unique_equipment_names = set(exercises['EquipmentName'].dropna().unique())

# Get the set of keys from the category_map
category_map_keys = set(equipment_category_map.keys())

# Find the equipment names that are not in the category_map keys
unmapped_equipment_names = unique_equipment_names - category_map_keys

# Count the number of unmapped equipment names
num_unmapped_equipment_names = len(unmapped_equipment_names)

print(f"Number of values in the column EquipmentName that don't belong to the keys of the category map: {num_unmapped_equipment_names}")

Number of values in the column EquipmentName that don't belong to the keys of the category map: 0


In [164]:
# Print the equipment names that are not in the category_map keys
print("Equipment names missing from the category map:")
for equipment in unmapped_equipment_names:
    if equipment.strip() not in category_map_keys:
        print(equipment.strip())


Equipment names missing from the category map:


In [165]:
# Strip whitespace from the specified columns
columns_to_strip = ['EquipmentName', 'ExerciseMusclesName', 'ExerciseBodyPartName', 'ExerciseName']
for column in columns_to_strip:
    exercises[column] = exercises[column].str.strip()

# Display the first few rows to verify the changes
exercises.head()

,CloudId,Gender,Age,PhysicalActivityMacroTypeName,ExerciseName,EquipmentName,DoneOnUTC,Duration_sec,Calories,MetsMin,ExerciseBodyPartName,ExerciseMusclesName,METs,EquipmentCategory
0,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,Custom exercise in time,Bike,2022-01-23 10:00:31.425000+00:00,215,30.0,5.6,LowerBody,NaN,NaN,Cardiovascular
1,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,GOAL exercise in time,Run,2022-01-23 10:09:01.153000+00:00,215,29.0,5.3,TotalBody,NaN,NaN,Cardiovascular
2,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Isotonic,Leg curl,Leg Curl Biostr,2022-01-23 10:28:49.724000+00:00,85,9.0,4.0,NaN,NaN,NaN,Strength Training
3,56a55ec8f237732fff2239735b469e346c8a5f6b,M,17,Cardio,GOAL exercise in time,Run,2022-01-23 08:52:44.240000+00:00,600,90.0,8.4,TotalBody,NaN,NaN,Cardiovascular
4,926bd1a7bbf9ccd40101befc6d409e2aa5979fd2,F,17,Cardio,GOAL exercise in time,Run,2022-01-23 17:13:38.840000+00:00,660,126.0,8.6,TotalBody,NaN,NaN,Cardiovascular


In [154]:
# Add a new column 'EquipmentCategory' to the exercises DataFrame
exercises['EquipmentCategory'] = exercises['EquipmentName'].map(equipment_category_map)

# Display the first few rows to verify the new column
exercises.head()

,CloudId,Gender,Age,PhysicalActivityMacroTypeName,ExerciseName,EquipmentName,DoneOnUTC,Duration_sec,Calories,MetsMin,ExerciseBodyPartName,ExerciseMusclesName,METs,EquipmentCategory
0,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,Custom exercise in time,Bike,2022-01-23 10:00:31.425000+00:00,215,30.0,5.6,LowerBody,NaN,NaN,Cardiovascular
1,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Cardio,GOAL exercise in time,Run,2022-01-23 10:09:01.153000+00:00,215,29.0,5.3,TotalBody,NaN,NaN,Cardiovascular
2,ed958cce9812f1b8607f8c8ab5836c1f450f858b,M,13,Isotonic,Leg curl,Leg Curl Biostr,2022-01-23 10:28:49.724000+00:00,85,9.0,4.0,NaN,NaN,NaN,Strength Training
3,56a55ec8f237732fff2239735b469e346c8a5f6b,M,17,Cardio,GOAL exercise in time,Run,2022-01-23 08:52:44.240000+00:00,600,90.0,8.4,TotalBody,NaN,NaN,Cardiovascular
4,926bd1a7bbf9ccd40101befc6d409e2aa5979fd2,F,17,Cardio,GOAL exercise in time,Run,2022-01-23 17:13:38.840000+00:00,660,126.0,8.6,TotalBody,NaN,NaN,Cardiovascular


In [ ]:
# Filter the exercises DataFrame for rows where EquipmentName is not null and EquipmentCategory is null
missing_equipment_category = exercises[(exercises['EquipmentName'].notnull()) & (exercises['EquipmentCategory'].isnull())]
# Display the filtered records
missing_equipment_category

,CloudId,Gender,Age,PhysicalActivityMacroTypeName,ExerciseName,EquipmentName,DoneOnUTC,Duration_sec,Calories,MetsMin,ExerciseBodyPartName,ExerciseMusclesName,METs,EquipmentCategory


In [156]:
exercises.isnull().sum()

CloudId                                0
Gender                                 0
Age                                    0
PhysicalActivityMacroTypeName          0
ExerciseName                           0
EquipmentName                          3
DoneOnUTC                              0
Duration_sec                           0
Calories                               3
MetsMin                          2456592
ExerciseBodyPartName              783304
ExerciseMusclesName               863014
METs                              870238
EquipmentCategory                      3
dtype: int64

### Store unique muscles for each exercise

In [157]:
exercises['ExerciseMusclesName'].value_counts().head(15)

ExerciseMusclesName
LateralsBack                            318728
Quadriceps                              298402
Pectorals                               272265
TotalBody                               260611
Abdominals                              222133
LowerBody                               156493
Deltoids                                110438
Hamstrings                              107163
Gluteus                                 102602
LowerBack                                75629
Gluteus|Quadriceps|Hamstrings|Calves     67488
Oblique                                  67341
Adductors                                62237
Triceps                                  61881
Biceps                                   57299
Name: count, dtype: int64

In [158]:
unique_exercise_muscles_name = exercises['ExerciseMusclesName'].nunique()
print(f"Unique ExerciseMusclesName: {unique_exercise_muscles_name}")

Unique ExerciseMusclesName: 268


In [ ]:
# Specify the column for which you want to extract unique values
column_name = 'ExerciseMusclesName'  # Change this to the desired column name
unique_values = exercises[column_name].dropna().unique()
unique_values_df = pd.DataFrame(unique_values, columns=[column_name])
unique_values_df.to_csv(f'unique_{column_name}.csv', index=False)

##### Store processed dataframe to file

In [174]:
# Store the processed exercises dataframe to a new CSV file
exercises.to_csv('data_cleaned/processed_exercises.csv', index=False)